In [2]:
import cv2
import os
import numpy as np
import random

# Directory containing all person folders
faces_dir = r"D:\Final_Semester_Project\AI_Attendance_System\AI_And_ML_Model\data\faces"

# Target number of images per class
target_images = 70

# Augmentation options
rotation_angles = [-15, -10, -5, 5, 10, 15]
brightness_factors = [0.7, 0.85, 1.15, 1.3]
zoom_factors = [0.9, 0.95, 1.05, 1.1]
shift_pixels = [-10, -5, 5, 10]  # horizontal/vertical shift in pixels
noise_std = 10  # standard deviation for Gaussian noise

def augment_image(img, method):
    """Apply one augmentation method to the image."""
    h, w = img.shape[:2]

    if method == "flip":
        return cv2.flip(img, 1)

    elif method == "rotate":
        angle = random.choice(rotation_angles)
        M = cv2.getRotationMatrix2D((w//2, h//2), angle, 1.0)
        return cv2.warpAffine(img, M, (w, h))

    elif method == "brightness":
        factor = random.choice(brightness_factors)
        return cv2.convertScaleAbs(img, alpha=factor, beta=0)

    elif method == "zoom":
        factor = random.choice(zoom_factors)
        new_w, new_h = int(w*factor), int(h*factor)
        resized = cv2.resize(img, (new_w, new_h))
        if factor > 1.0:
            # crop center
            start_x = (new_w - w)//2
            start_y = (new_h - h)//2
            return resized[start_y:start_y+h, start_x:start_x+w]
        else:
            # pad with black
            padded = np.zeros_like(img)
            start_x = (w - new_w)//2
            start_y = (h - new_h)//2
            padded[start_y:start_y+new_h, start_x:start_x+new_w] = resized
            return padded

    elif method == "shift":
        tx = random.choice(shift_pixels)
        ty = random.choice(shift_pixels)
        M = np.float32([[1, 0, tx], [0, 1, ty]])
        return cv2.warpAffine(img, M, (w, h))

    # elif method == "noise":
    #     noise = np.random.normal(0, noise_std, img.shape).astype(np.uint8)
    #     noisy = cv2.add(img, noise)
    #     return noisy

    else:
        return img  # no augmentation

# Available augmentation methods
methods = ["flip", "rotate", "brightness", "zoom", "shift", "noise"]

# Loop through each person folder
for person_name in os.listdir(faces_dir):
    person_path = os.path.join(faces_dir, person_name)
    if not os.path.isdir(person_path):
        continue

    images = [f for f in os.listdir(person_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    num_images = len(images)

    print(f"Processing {person_name}: {num_images} images")

    if num_images >= target_images:
        continue

    needed = target_images - num_images
    idx = 0

    while needed > 0:
        img_name = images[idx % len(images)]
        img_path = os.path.join(person_path, img_name)
        img = cv2.imread(img_path)
        if img is None:
            idx += 1
            continue

        # Randomly pick one augmentation method
        method = random.choice(methods)
        aug_img = augment_image(img, method)

        out_name = f"{os.path.splitext(img_name)[0]}_aug{num_images}.jpg"
        out_path = os.path.join(person_path, out_name)
        cv2.imwrite(out_path, aug_img)

        num_images += 1
        needed -= 1
        idx += 1

    print(f"→ After augmentation: {num_images} images")

print("✅ Dataset balanced with realistic, single-step augmentations!")


Processing Aishwarya_Rai: 82 images
Processing Akshay_Kumar: 37 images
→ After augmentation: 70 images
Processing Alia_Bhatt: 60 images
→ After augmentation: 70 images
Processing Allu_Arjun: 45 images
→ After augmentation: 70 images
Processing Amitabh_Bachchan: 52 images
→ After augmentation: 70 images
Processing Angelina_Jolie: 75 images
Processing Ariana_Grande: 66 images
→ After augmentation: 70 images
Processing Barack_Obama: 69 images
→ After augmentation: 70 images
Processing Bhuwan_K.C: 57 images
→ After augmentation: 70 images
Processing Billie_Eilish: 79 images
Processing Bill_Gates: 69 images
→ After augmentation: 70 images
Processing Brad_Pitt: 75 images
Processing Chris_Evans: 77 images
Processing Cristiano_Ronaldo: 66 images
→ After augmentation: 70 images
Processing Deepika_Padukone: 77 images
Processing Drake: 65 images
→ After augmentation: 70 images
Processing Dwayne_Johnson: 86 images
Processing Ed_Sheeran: 81 images
Processing Emma_Watson: 96 images
Processing Hrithi